<a href="https://colab.research.google.com/github/BruceSteven/Text-Dectection-/blob/main/Baseline_CRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Môi Trường



In [ ]:
#kiem tra gpu
!nvidia-smi

In [ ]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

##Clone source code từ github và set-up các tài nguyên cần thiết

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!git clone https://github.com/CS-UIT-AI-CLUB/CRAFT

In [ ]:
%cd CRAFT

In [ ]:
# install các package cần thiết
!pip install pillow==6.2.2

In [ ]:
!pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install water 

In [ ]:
#cac thu muc de chua file test
!mkdir result
!mkdir results
!mkdir tested_image

In [ ]:
%cd /content/drive/MyDrive/CRAFT/pretrain
#tai cac file can thiet cho training
!gdown --id 1-RM4shR-oj0cWO-9mthKZKnyF-qy1OZW -O 91.pth
!gdown --id 1wQDetvezJGp6eBVK7_yInOzZdpoD3k7j -O vgg16_bn-6c64b313.pth

##Tải dữ liệu

In [ ]:
%cd /content/drive/MyDrive
!gdown --id 13_4EqHuOV2T2Sq_oCKFYJMqLgSCoBjHM -O train.zip
!unzip train.zip 

In [ ]:
#rename thư mục theo định dạng đúng
import os
os.rename('./train/images','./train/ch4_training_images')
os.rename('./train/labels','./train/ch4_training_localization_transcription_gt')

In [ ]:
#thay doi dinh dang data
%cd /content/drive/MyDrive/train/ch4_training_localization_transcription_gt
directory = os.listdir('/content/drive/MyDrive/train/ch4_training_localization_transcription_gt')

for file in directory:
	name, ext = os.path.splitext(file)
	os.rename(file, 'gt_' + name + ext)

## Bắt đầu train

In [ ]:
%cd /content/drive/MyDrive/CRAFT

In [ ]:
!python trainic15data.py

##Test

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
#tai test data
!gdown --id 14Wf3YMoXT9LcOQWaJbRd3Or2pXMUHO3C -O test.zip
!unzip test.zip

In [ ]:
%cd /content/drive/MyDrive/CRAFT/results

In [ ]:
!gdown --id 1-RM4shR-oj0cWO-9mthKZKnyF-qy1OZW

In [ ]:
%cd /content/drive/MyDrive/CRAFT

In [ ]:
!python test.py

In [ ]:
#chuyen dinh dang file test 
import os
%cd /content/drive/MyDrive/CRAFT/tested_image
directory = os.listdir('/content/drive/MyDrive/CRAFT/tested_image')
count = 0
text_file = open("aaaa.txt","a")
for file in directory:
	if file.endswith(".txt"):
		count = count + 1
		with open(file) as f:
			content = f.readlines()
	# you may also want to remove whitespace characters like `\n` at the end of each line
		content = [x.strip() for x in content] 
		for i in range(len(content)) :
			text_file.write("{0},{1}\n".format(file,content[i]))
   
print(count)

In [ ]:
%cd /content/drive/MyDrive/CRAFT/result
with open('/content/drive/MyDrive/CRAFT/tested_image/aaaa.txt') as f:
  contents = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
contents = [x.strip() for x in contents] 

for i in range(len(contents)):
  line = contents[i].split(',')
  file_name = line[0]
  line[0] = file_name[4:-4] + ".jpg"
  text_file = open('test.txt', "a")
  text_file.write("{0},{1},{2},{3},{4},{5},{6},{7},{8}\n".format(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8]))

##Submission

In [ ]:
import math, requests

def submit_prediction(df, comment='', **kwargs):
    TOKEN='f9f0fe020c715fcc74d65e919507405f1ff39dddfd93556c23513e77abb5b6dbe478fc7a6d00f2fc78f1b138fd0f9338b4e485db6e7406c02954bc92e2c7b0a2'
    URL='http://submission.mmlab.uit.edu.vn/api/submissions'
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': open(df, 'rb')},data={'comment':comment})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)
        
submit_prediction('/content/drive/MyDrive/CRAFT/result/test.txt', comment='my submission')